## Modeling Facade

In [27]:
import pandas as pd
import numpy as np
from pathlib import Path

from models.baseline_ctr import BaselineCTRRegressor
from models.multimodal_ctr import MultimodalCTRRegressor

DATA_PATH = Path("data/data.csv")
IMAGES_ROOT = "images"  # folder with your creatives

In [ ]:
df = pd.read_csv(DATA_PATH)

df["time"] = pd.to_datetime(df["time"])
df = df.sort_values("time").reset_index(drop=True)

df.head()

/var/folders/yx/b_pyg98x1dj97pbflpck32d80000gn/T/ipykernel_6595/3827788563.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["time"] = pd.to_datetime(df["time"])


,campaign_item_id,no_of_days,time,ext_service_id,ext_service_name,creative_id,search_tags,template_id,landing_page,advertiser_id,...,clicks,stats_currency,currency_code,exchange_rate,media_cost_usd,search_tag_cat,cmi_currency_code,timezone,weekday_cat,keywords
0,2733,7,2022-05-01,128,Facebook Ads,1000,#The Power of X,90.0,https://www.abcjewelry.com/collections/boho-je...,4756,...,8,SGD,SGD,1,14.058514,Others,SGD,Asia/Singapore,week_end,delicate bracelets
1,2766,23,2022-05-01,16,DV360,13720,#Embrace Your Individuality with X,NaN,https://www.abcjewelry.com/women/affordable-je...,5209,...,23,USD,USD,1,16.368097,Others,USD,America/New_York,week_end,trendy and timeless jewelry
2,3543,9,2022-05-01,16,DV360,13530,#The Power of X,90.0,https://www.abcjewelry.com/collections/hoop-ea...,6370,...,47,INR,INR,1,24.069760,Others,INR,Asia/Kolkata,week_end,party jewelry
3,3389,0,2022-05-01,16,DV360,15500,#The Power of X,NaN,https://www.abcjewelry.com/collections/unique-...,6292,...,9,INR,INR,1,0.740751,Others,INR,Asia/Kolkata,week_end,long necklaces
4,3543,16,2022-05-01,4,Google Ads,13525,#Be Bold. Be X,90.0,https://www.abcjewelry.com/collections/gemston...,6370,...,1003,INR,INR,1,168.393050,Others,INR,Asia/Kolkata,week_end,long necklaces


In [29]:
test_size = 0.2
split_idx = int(len(df) * (1.0 - test_size))

df_train = df.iloc[:split_idx].reset_index(drop=True)
df_test = df.iloc[split_idx:].reset_index(drop=True)

df_train.shape, df_test.shape

((58089, 30), (14523, 30))

In [30]:
baseline = BaselineCTRRegressor()
baseline.fit(df_train)
baseline_metrics = baseline.evaluate(df_test)

print("Baseline tabular-only metrics:")
for k, v in baseline_metrics.items():
    print(f"{k}: {v:.6f}")


Baseline tabular-only metrics:
rmse: 0.004463
mae: 0.001612
r2: 0.816840


In [31]:
multimodal = MultimodalCTRRegressor(images_root=IMAGES_ROOT, device="cpu")
multimodal.fit(df_train)
multimodal_metrics = multimodal.evaluate(df_test)

print("Multimodal (tabular + image) metrics:")
for k, v in multimodal_metrics.items():
    print(f"{k}: {v:.6f}")


ValueError: You have to specify input_ids

In [11]:
df_sample = df_test.sample(n=5, random_state=42).reset_index(drop=True)

baseline_pred = baseline.predict(df_sample)
multimodal_pred = multimodal.predict(df_sample)

comparison = df_sample[["campaign_item_id", "creative_id", "clicks", "impressions"]].copy()
comparison["true_ctr"] = df_sample["clicks"] / df_sample["impressions"].clip(lower=1)
comparison["baseline_ctr_pred"] = baseline_pred
comparison["multimodal_ctr_pred"] = multimodal_pred

comparison

ValueError: You have to specify input_ids